# models for opcode frequency

## Import Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import time
import json
import pandas as pd
from pathlib import Path
import joblib

import optuna
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.multioutput import MultiOutputClassifier


## Const

In [3]:
PATH = Path.cwd().parents[1]
DATA_PATH = os.path.join(PATH, 'data/processed')
MODEL_PATH = os.path.join(PATH, 'models')

In [4]:
with open(os.path.join(DATA_PATH, 'feature-opcode-freq_list.json')) as f:
    feature_list = json.load(f)

with open(os.path.join(DATA_PATH, 'labels-opcode-freq.json')) as f:
    labels = json.load(f)

In [5]:
train_df = pd.read_csv(os.path.join(DATA_PATH, 'train-opcode-freq.csv'))
test_df = pd.read_csv(os.path.join(DATA_PATH, 'test-opcode-freq.csv'))

X_train = train_df[feature_list]
y_train = train_df[labels]

X_test = test_df[feature_list]
y_test = test_df[labels]

In [6]:
X_train.head()

,PUSH1,MSTORE,CALLDATASIZE,LT,PUSH2,JUMPI,CALLDATALOAD,PUSH29,SWAP1,DIV,...,UNKNOWN_0xc6,UNKNOWN_0xe1,INVALID_0x70,PUSH30,DUP16,UNKNOWN_0x2b,UNKNOWN_0xd8,INVALID_0x7a,UNKNOWN_0xf9,INVALID_0x7f
0,1,0,2,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,16,2,2,0,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,40,5,5,2,32,10,2,0,7,0,...,0,0,0,0,1,1,0,0,0,0
3,53,7,5,2,43,9,2,0,8,0,...,0,0,0,0,0,0,0,0,0,0
4,178,32,7,7,134,41,6,0,66,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
y_train.head()

,mint,leak,limit
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


## Traditional Machine Learning Models

In [8]:
models = {
    "Logistic Regression": MultiOutputClassifier(LogisticRegression(max_iter=1000)),
    "Random Forest": MultiOutputClassifier(RandomForestClassifier()),
    "Gradient Boosting": MultiOutputClassifier(GradientBoostingClassifier()),
    "AdaBoost": MultiOutputClassifier(AdaBoostClassifier()),
    "SVM (Linear)": MultiOutputClassifier(SVC(kernel="linear")),
    "KNN": MultiOutputClassifier(KNeighborsClassifier()),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "MLP Classifier": MultiOutputClassifier(MLPClassifier(max_iter=300)),
    "XGBoost": MultiOutputClassifier(XGBClassifier(use_label_encoder=False, eval_metric='logloss')),
    "LightGBM": MultiOutputClassifier(LGBMClassifier()),
    "DecisionTree": MultiOutputClassifier(DecisionTreeClassifier())
}

In [9]:
results = []

for name, model in models.items():
    start = time.time()
    model.fit(X_train, y_train)
    end = time.time()

    y_pred = model.predict(X_test)

    results.append({
        "Classifier": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, average="macro"),  # change to 'macro' if multi-class
        "Recall": recall_score(y_test, y_pred, average="macro"),
        "F1-Score": f1_score(y_test, y_pred, average="macro"),
        "Training Time": round(end - start, 3)
    })

[LightGBM] [Info] Number of positive: 39, number of negative: 98
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000352 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1889
[LightGBM] [Info] Number of data points in the train set: 137, number of used features: 94
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.284672 -> initscore=-0.921406
[LightGBM] [Info] Start training from score -0.921406
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

### Result

In [10]:
df = pd.DataFrame(results)
df.sort_values(by="F1-Score", ascending=False, inplace=True)
df

,Classifier,Accuracy,Precision,Recall,F1-Score,Training Time
9,LightGBM,0.428571,0.888889,0.388889,0.484127,0.084
5,KNN,0.285714,0.571429,0.407407,0.475000,0.007
8,XGBoost,0.428571,0.805556,0.388889,0.471306,0.509
0,Logistic Regression,0.285714,0.527778,0.351852,0.422222,0.497
10,DecisionTree,0.214286,0.654762,0.351852,0.398810,0.016
3,AdaBoost,0.285714,0.477778,0.333333,0.387302,0.296
2,Gradient Boosting,0.285714,0.694444,0.277778,0.360195,0.762
1,Random Forest,0.428571,0.833333,0.240741,0.349784,0.336
4,SVM (Linear),0.285714,0.416667,0.277778,0.318627,0.103
7,MLP Classifier,0.214286,0.305556,0.185185,0.206349,0.863


### Tuning

In [11]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 500),
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "num_leaves": trial.suggest_int("num_leaves", 3, 255),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1
    }

    model = MultiOutputClassifier(LGBMClassifier(**params))

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred, average="macro", zero_division=0)

In [12]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("Tuned LGBMClassifier (MultiOutput):")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average="macro", zero_division=0))
print("Recall:", recall_score(y_test, y_pred, average="macro", zero_division=0))
print("F1 Score:", f1_score(y_test, y_pred, average="macro", zero_division=0))

[I 2025-07-12 11:31:58,712] A new study created in memory with name: no-name-6933e261-4a85-43b0-af01-07de9e812e9d
[I 2025-07-12 11:31:59,062] Trial 0 finished with value: 0.26666666666666666 and parameters: {'n_estimators': 104, 'learning_rate': 0.0032999242371406423, 'max_depth': 5, 'num_leaves': 115, 'min_child_samples': 23, 'subsample': 0.748377228674205, 'colsample_bytree': 0.6099281713050817, 'reg_alpha': 8.793892192615997e-08, 'reg_lambda': 1.225559812056827e-07}. Best is trial 0 with value: 0.26666666666666666.


[LightGBM] [Info] Number of positive: 39, number of negative: 98
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002340 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1878
[LightGBM] [Info] Number of data points in the train set: 137, number of used features: 91
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.284672 -> initscore=-0.921406
[LightGBM] [Info] Start training from score -0.921406
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

[I 2025-07-12 11:31:59,318] Trial 1 finished with value: 0.3841269841269841 and parameters: {'n_estimators': 350, 'learning_rate': 0.06591592108633305, 'max_depth': 15, 'num_leaves': 162, 'min_child_samples': 44, 'subsample': 0.7298690872300824, 'colsample_bytree': 0.9999032555231511, 'reg_alpha': 1.5503254174626225e-06, 'reg_lambda': 0.00012478193830898964}. Best is trial 1 with value: 0.3841269841269841.
[I 2025-07-12 11:31:59,443] Trial 2 finished with value: 0.0 and parameters: {'n_estimators': 400, 'learning_rate': 0.00020328415881537752, 'max_depth': 13, 'num_leaves': 242, 'min_child_samples': 42, 'subsample': 0.9095849995249684, 'colsample_bytree': 0.9949599918305516, 'reg_alpha': 0.00014425496082959074, 'reg_lambda': 2.577363498196816e-06}. Best is trial 1 with value: 0.3841269841269841.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 39, number of negative: 98
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000275 seconds.
You ca

[I 2025-07-12 11:31:59,879] Trial 3 finished with value: 0.0 and parameters: {'n_estimators': 370, 'learning_rate': 1.3007962261002906e-06, 'max_depth': 10, 'num_leaves': 247, 'min_child_samples': 16, 'subsample': 0.5772875271537765, 'colsample_bytree': 0.7755882982617321, 'reg_alpha': 0.00012146566667649076, 'reg_lambda': 1.4931863313128469e-05}. Best is trial 1 with value: 0.3841269841269841.


...

[LightGBM] [Info] Number of positive: 39, number of negative: 98
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000374 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1913
[LightGBM] [Info] Number of data points in the train set: 137, number of used features: 100
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.284672 -> initscore=-0.921406
[LightGBM] [Info] Start training from score -0.921406
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

[I 2025-07-12 11:32:07,889] Trial 48 finished with value: 0.0 and parameters: {'n_estimators': 212, 'learning_rate': 0.000545958092139341, 'max_depth': 13, 'num_leaves': 20, 'min_child_samples': 28, 'subsample': 0.6684670664428018, 'colsample_bytree': 0.7569702151168766, 'reg_alpha': 0.13979419787419872, 'reg_lambda': 8.274293602350496e-06}. Best is trial 24 with value: 0.5793650793650793.


[LightGBM] [Info] Number of positive: 39, number of negative: 98
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001631 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1864
[LightGBM] [Info] Number of data points in the train set: 137, number of used features: 88
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.284672 -> initscore=-0.921406
[LightGBM] [Info] Start training from score -0.921406
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2025-07-12 11:32:08,108] Trial 49 finished with value: 0.0 and parameters: {'n_estimators': 169, 'learning_rate': 0.00012951608828948337, 'max_depth': 13, 'num_leaves': 41, 'min_child_samples': 7, 'subsample': 0.6375624202443606, 'colsample_bytree': 0.9180533037815783, 'reg_alpha': 0.001916903645218976, 'reg_lambda': 2.5638701327825622e-05}. Best is trial 24 with value: 0.5793650793650793.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [13]:
joblib.dump(model, os.path.join(MODEL_PATH, f'best_lgbm-moc_model_on_crpwarner_opcode_freq.pkl'))

['/Users/napatcholthaipanich/Dev/master/dissertation/workspace/ml/models/best_lgbm-moc_model_on_crpwarner_opcode_freq.pkl']

### K-Fold (K=3)

In [14]:
NUM_FOLDS = 3
results = []
best_model = MultiOutputClassifier(LGBMClassifier(**study.best_params, random_state=42))
best_f1 = 0
best_fold = 0

In [15]:
for fold in range(NUM_FOLDS):
    print(f"=========== Fold-{fold} ===========")
    train_path = os.path.join(DATA_PATH, f'train_fold_{fold}-opcode-freq.csv')
    val_path = os.path.join(DATA_PATH, f'val_fold_{fold}-opcode-freq.csv')

    train_df = pd.read_csv(train_path)
    val_df   = pd.read_csv(val_path)

    X_train = train_df[feature_list]
    y_train = train_df[labels]

    X_val = val_df[feature_list]
    y_val = val_df[labels]

    # Train model
    model = MultiOutputClassifier(LGBMClassifier(**study.best_params, random_state=42))
    model.fit(X_train, y_train)

    # Evaluate
    y_pred = model.predict(X_val)
    report = classification_report(y_val, y_pred, target_names=labels, output_dict=True)
    acc = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average="macro", zero_division=0)

    results.append({'fold': fold, 'accuracy': acc, 'report': report})
    print(f"Accuracy: {acc}")
    print("Precision:", precision_score(y_val, y_pred, average="macro", zero_division=0))
    print("Recall:", recall_score(y_val, y_pred, average="macro", zero_division=0))
    print("F1 Score:", f1)

    if best_f1 < f1:
        best_model = model
        best_fold = fold
## Step 6: Average Performance Summary
print("\n===== Overall Summary =====")
avg_acc = sum([r['accuracy'] for r in results]) / NUM_FOLDS
print(f"Average Accuracy: {avg_acc:.4f}")

# Save model
joblib.dump(best_model, os.path.join(MODEL_PATH, f'best_lgbm-ovr_model_on_crpwarner_opcode_freq_from_fold{best_fold}.pkl'))

=========== Fold-0 ===========
[LightGBM] [Info] Number of positive: 13, number of negative: 33
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000505 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1104
[LightGBM] [Info] Number of data points in the train set: 46, number of used features: 182
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.282609 -> initscore=-0.931558
[LightGBM] [Info] Start training from score -0.931558
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

['/Users/napatcholthaipanich/Dev/master/dissertation/workspace/ml/models/best_lgbm-ovr_model_on_crpwarner_opcode_freq_from_fold2.pkl']